In [1]:
import os
import sys
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.controller.combined_robot import *
from pkg.project_config import *

In [2]:
CONNECT_INDY = False

MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

In [3]:
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                MOBILE_IP),
    RobotConfig(1, RobotType.indy7, ((0.172,0,0.439), (0,0,np.pi)),
                INDY_IP, root_on="kmb0_platform", specs={"no_sdk":True})]
              , connection_list=[False, CONNECT_INDY])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
# xyz_rpy_robots = {"kmb0": ((0,0,0), (0,0,0)), "indy1": ((0,0,0), (0,0,np.pi))}
# crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
gscene.show_pose(-crob.home_pose)

connection command:
kmb0: False
indy1: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
# gscene.show_pose(-crob.home_pose + ([0.2,0.2,np.pi/3,]+[0]*6))

## set workspace

In [5]:
gscene.set_workspace_boundary( -5, 5, -4, 4, -0.1, 1.75)

Please create a subscriber to the marker


## define robot and planning scene

In [6]:
from pkg.planning.scene import *
from demo_utils.environment import *

CHAIR_NAME = "chair0"
CHAIR_DIM = (0.4,0.4,0.5)

MOBILE_NAME = "kmb0"
HOLD_LINK = "kmb0_platform"
HOLD_XYZ = (0.445 + 0.35, 0, CHAIR_DIM[2]/2)
HOLD_RPY = (0, -np.pi/2, -np.pi/2)
HOLD_NAME = "hold0"

ROBOT_NAME = "indy1"
TOOL_LINK = "indy1_tcp"
TOOL_NAME = "brush_face"

HOME_POSE = -crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

gtems_robot = s_builder.add_robot_geometries(color=(0, 1, 0, 0.5), display=True, collision=True)
viewpoint = add_cam(gscene, tool_link=TOOL_LINK)
add_indy_tool_kiro(gscene, tool_link=TOOL_LINK, face_name=TOOL_NAME, zoff=-0.04)

pscene = PlanningScene(gscene, combined_robot=crob)

## planning pipeline

In [7]:

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)


from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [8]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.motion_filters = [GraspChecker(pscene)]

   Use a production WSGI server instead.
 * Debug mode: off


## Add action points

In [9]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

In [10]:
CLEARANCE = 0.001
# floor
gscene.create_safe(gtype=GEOTYPE.BOX, name="floor", link_name="base_link",
                   dims=(10,8,0.1), center=(0,0,-0.05-CLEARANCE), 
                   rpy=(0,0,0), color=(0.9, 0.9, 0.9, 0.9), display=True,
                   collision=True, fixed=True)
floor = pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane, point=None)

# brush_face = pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, 
#                                   point=(-gscene.NAME_DICT['brush_face'].dims[0]/2,0,0), rpy=(0,np.pi/2*1,0))

# waypoint
WP_DIMS = (0.6,0.4,0.1)
gscene.create_safe(gtype=GEOTYPE.BOX, name="wayframer", link_name=HOLD_LINK,
                   dims=WP_DIMS, center=(0,0,WP_DIMS[2]/2), rpy=(0,0,0), color=(1, 0, 0, 0.5), display=True,
                   collision=False, fixed=True)
wayframer = pscene.create_binder(bname="wayframer", gname="wayframer", _type=WayFramer, 
                                 point=(0,0,-WP_DIMS[2]/2), rpy=(0,0,0))


waypoints = [((2-1,-1,0),(0,0,np.pi/2)), ((2+1,-1,0),(0,0,np.pi/2)), ((2+1,1,0),(0,0,-np.pi/2)), ((2-1,1,0),(0,0,-np.pi/2))]
wp_list = []
for i_w, (xyz, rpy) in enumerate(waypoints):
    wp_list.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="wp_{}".format(i_w), link_name="base_link",
                           dims=WP_DIMS, center=xyz, 
                           rpy=rpy, color=(0.6, 0.0, 0.0, 0.5), display=True, collision=False, fixed=True, parent="floor")
    )
wp_task = pscene.create_subject(oname="waypoints", gname="floor", _type=WayopintTask, 
                                action_points_dict={wp.name: WayFrame(wp.name, wp, [0,0,0.05], [0,0,0]) for wp in wp_list})


# chair holder
gscene.create_safe(gtype=GEOTYPE.SPHERE, name=HOLD_NAME, link_name=HOLD_LINK,
                   dims=(0.01,) * 3, center=HOLD_XYZ, rpy=HOLD_RPY, color=(1, 0, 0, 0.5), display=True,
                   collision=False,
                   fixed=True)
holder = pscene.create_binder(bname=HOLD_NAME, gname=HOLD_NAME, _type=Gripper2Tool, point=(0, 0, 0), rpy=(0, 0, 0))
holder.redundancy = {}

## Add chair

In [11]:
chair_loc = (1, -1, CHAIR_DIM[2]/2)

obs_all = 3
for i_c in range(obs_all):
    CHAIR_NAME_CUR = CHAIR_NAME[:-1]+"%01d"%i_c
    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name=CHAIR_NAME_CUR, link_name="base_link",
                       dims=CHAIR_DIM, center=np.add(chair_loc, [0,0.75*i_c, 0]), rpy=(0,0,0),
                       color=(1,1,0,1), display=True, collision=True, fixed=False)
    chair = pscene.create_subject(oname=CHAIR_NAME_CUR, gname=CHAIR_NAME_CUR, _type=CylinderObject, 
                                  GRASP_WIDTH_MIN=CHAIR_DIM[0]-0.1, GRASP_WIDTH_MAX=CHAIR_DIM[0]+0.1, 
                                  GRASP_DEPTH_MIN=CHAIR_DIM[0]/2, GRASP_DEPTH_MAX=CHAIR_DIM[0]/2)
#         chair.action_points_dict[CHAIR_NAME_CUR+"_side_g"].redundancy = {'u': (-np.pi, np.pi)}
    chair.action_points_dict[CHAIR_NAME_CUR+"_side_g"].redundancy = {'u': (-0, 0)}

## [Test] pick and place chair

In [12]:
initial_state = pscene.initialize_state(-crob.home_pose)

In [13]:
initial_state = pscene.initialize_state(-crob.home_pose)
goal_nodes = [('floor',)*3+(4,)]

In [14]:
from pkg.planning.task.rrt import TaskRRT

tplan = TaskRRT(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

ppline.search(initial_state, goal_nodes, verbose=True,
              timeout=1, timeout_loop=100, multiprocess=False,
              add_homing=True, home_pose=HOME_POSE)
schedules = ppline.tplan.find_schedules(False)
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
ppline.play_schedule(snode_schedule)


try: 0 - ('floor', 'floor', 'floor', 0)->('floor', 'floor', 'floor', 1)
Motion Filer Failure: GraspChecker
result: 0 - ('floor', 'floor', 'floor', 0)->('floor', 'floor', 'floor', 1) = fail
try: 0 - ('floor', 'floor', 'floor', 0)->('floor', 'floor', 'floor', 1)
Motion Filer Failure: GraspChecker
result: 0 - ('floor', 'floor', 'floor', 0)->('floor', 'floor', 'floor', 1) = fail
try: 0 - ('floor', 'floor', 'floor', 0)->('floor', 'floor', 'hold0', 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 'floor', 'floor', 0)->('floor', 'floor', 'hold0', 0) = success
branching: 0->1 (0.06/100.0 s, steps/err: 7(46.6089248657 ms)/0.000950999417782)
try: 1 - ('floor', 'floor', 'hold0', 0)->('floor', 'floor', 'hold0', 1)
Motion Filer Failure: GraspChecker
result: 1 - ('floor', 'floor', 'hold0', 0)->('floor', 'floor', 'hold0', 1) = fail
try: 0 - ('floor', 'floor', 'floor', 0)->('floor', 'hold0', 'floor', 0)
Motion Filer Failure: GraspChecker
result: 0 - ('floor', 'floor', 'floo

transition motion tried: True
result: 4 - ('floor', 'floor', 'floor', 0)->('floor', 'floor', 'hold0', 0) = success
branching: 4->13 (0.87/100.0 s, steps/err: 57(64.6619796753 ms)/0.00118834787618)
try: 13 - ('floor', 'floor', 'hold0', 0)->('floor', 'floor', 'floor', 0)
try transition motion
transition motion tried: True
result: 13 - ('floor', 'floor', 'hold0', 0)->('floor', 'floor', 'floor', 0) = success
branching: 13->14 (0.93/100.0 s, steps/err: 32(66.1120414734 ms)/0.000800811471073)
try: 13 - ('floor', 'floor', 'hold0', 0)->('floor', 'floor', 'floor', 0)
Motion Filer Failure: GraspChecker
result: 13 - ('floor', 'floor', 'hold0', 0)->('floor', 'floor', 'floor', 0) = fail
try: 12 - ('floor', 'floor', 'floor', 1)->('hold0', 'floor', 'floor', 1)
try transition motion
transition motion tried: True
result: 12 - ('floor', 'floor', 'floor', 1)->('hold0', 'floor', 'floor', 1) = success
branching: 12->15 (1.03/100.0 s, steps/err: 97(90.9240245819 ms)/0.00175882695233)
try: 15 - ('hold0', 'fl

transition motion tried: True
result: 5 - ('floor', 'floor', 'floor', 1)->('floor', 'hold0', 'floor', 1) = success
branching: 5->27 (1.93/100.0 s, steps/err: 73(55.2651882172 ms)/0.00177462182621)
try: 27 - ('floor', 'hold0', 'floor', 1)->('floor', 'floor', 'floor', 1)
try transition motion
transition motion tried: True
result: 27 - ('floor', 'hold0', 'floor', 1)->('floor', 'floor', 'floor', 1) = success
branching: 27->28 (2.01/100.0 s, steps/err: 44(73.0321407318 ms)/0.00131764150246)
try: 18 - ('floor', 'hold0', 'floor', 0)->('floor', 'hold0', 'floor', 1)
Motion Filer Failure: GraspChecker
result: 18 - ('floor', 'hold0', 'floor', 0)->('floor', 'hold0', 'floor', 1) = fail
try: 21 - ('floor', 'hold0', 'floor', 1)->('floor', 'hold0', 'floor', 2)
Motion Filer Failure: GraspChecker
result: 21 - ('floor', 'hold0', 'floor', 1)->('floor', 'hold0', 'floor', 2) = fail
try: 17 - ('floor', 'floor', 'floor', 0)->('hold0', 'floor', 'floor', 0)
try transition motion
transition motion tried: True
re

('hold0', 'floor', 'floor', 1)->('floor', 'floor', 'floor', 1)
('floor', 'floor', 'floor', 1)->('floor', 'floor', 'floor', 2)
('floor', 'floor', 'floor', 2)->('floor', 'floor', 'hold0', 2)
('floor', 'floor', 'hold0', 2)->('floor', 'floor', 'floor', 2)
('floor', 'floor', 'floor', 2)->('floor', 'floor', 'floor', 3)
('floor', 'floor', 'floor', 3)->('floor', 'floor', 'floor', 4)
('floor', 'floor', 'floor', 4)->('floor', 'floor', 'floor', 4)


In [14]:
ppline.play_schedule(snode_schedule)

(0,)->(0,)
(0,)->(1,)
(1,)->(2,)
(2,)->(3,)
(3,)->(4,)
(4,)->(4,)


In [25]:
# from pkg.planning.pddlstream.plan_rnb import solve_in_pddlstream

# goal_pairs=[(CHAIR_NAME, 'goal'), (CHAIR_NAME[:-1]+"1", "floor"), (CHAIR_NAME[:-1]+"2", "floor")]
# TIMEOUT_MOTION, MAX_TIME, MAX_ITER, MAX_SKELETONS = 1, 100, 100, 30
# GRASP_SAMPLE, STABLE_SAMPLE, SHOW_STATE, SEARCH_SAMPLE_RATIO = 100, 100, False, 100


# res, plan, log_dict = solve_in_pddlstream(pscene, mplan, MOBILE_NAME, HOLD_NAME, HOME_POSE, goal_pairs,
#                         TIMEOUT_MOTION, MAX_TIME, MAX_ITER, MAX_SKELETONS,
#                         GRASP_SAMPLE, STABLE_SAMPLE, SHOW_STATE, SEARCH_SAMPLE_RATIO,
#                         use_pybullet_gui=False, USE_MOVEIT_IK=True)